In [176]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from matplotlib.backends.backend_pdf import PdfPages
from statsmodels.tsa.arima.model import ARIMAResults
from pmdarima.arima import AutoARIMA
from statsmodels.tsa.seasonal import STL
from autots import AutoTS
from IPython.display import Image
import sys
import logging
from autots.models.model_list import model_lists
import json
from contextlib import contextmanager
import os
from helpers.constants import START_FORECASTING_YEAR, FORECASTING_LENGTH
from helpers.auto_arima_modelling import forecast_arima
from helpers.autots_modelling import forecast_auto_ts

# 1. Forecasting values

<b> Important Assumption:</b>

1. For models where testing was impossible due to:
- Insufficient data for testing.
- Insufficient data for training.
  
2. For models where the prediction was constant (due to errors in forecasting):

It is assumed that we couldn't test a model and therefore a models cannot be chosen for forecasting. This assumption persists even if there is now sufficient training data available (since we are not partitioning for testing), or if the prediction would not have been constant under normal forecasting conditions.

<b> Action: </b>

In such cases, the LAST AVAILABLE VALUE will be chosen for forecasting.

### Read all data

In [177]:
#Read FAO data

# Define the path to the CSV file
csv_file = 'data/AOSTAT_animal_2022_cleaned.csv'

# Load the CSV file into a DataFrame
data_df = pd.read_csv(csv_file)

# Count unique combinations of 'Area' and 'Item'
unique_combinations = data_df.groupby(['Area', 'Item']).size().reset_index(name='Count')

# Display the unique combinations and their counts
print(unique_combinations)

             Area          Item  Count
0     Afghanistan         Asses     62
1     Afghanistan        Camels     62
2     Afghanistan        Cattle     62
3     Afghanistan         Goats     62
4     Afghanistan        Horses     62
...           ...           ...    ...
1163     Zimbabwe        Cattle     62
1164     Zimbabwe         Goats     62
1165     Zimbabwe        Horses     62
1166     Zimbabwe         Sheep     62
1167     Zimbabwe  Swine / pigs     62

[1168 rows x 3 columns]


In [178]:
# Read all excel outputfiles

# Assuming your CSV file is in the current directory or you provide the full path
excel_file = 'ARIMA_evaluation_results_all.xlsx'
#Read the CSV file into a DataFrame
df = pd.read_excel(excel_file)

#obtain only the results where it could be modelled with an AutoARIMA and also where the data had an error like not enough data, not training data...
df_filtered = df[((df['Flag results'] == 'AutoARIMA') & (df['Predictions'] == 'MODEL')) | (df['Flag results'] != 'AutoARIMA')]

arima_results_df = df_filtered[['Country', 'Animal Type', 'Flag results', 'Model_type', 'Flag', 'Predictions']]

# Count the occurrences of each unique value in 'Flag results'
flag_results_counts = arima_results_df['Flag results'].value_counts()

# Display the counts
print(flag_results_counts)


Flag results
AutoARIMA     599
NoTestData     48
No Data        45
constant        1
Name: count, dtype: int64


In [179]:

# Assuming your CSV file is in the current directory or you provide the full path
excel_file = 'AUTOTS_evaluation_results_all.xlsx'
#Read the CSV file into a DataFrame
df = pd.read_excel(excel_file)
df_filtered = df[((df['Flag results'] == 'AutoTS') & (df['Predictions'] == 'MODEL'))]

autoTS_results_df = df_filtered[['Country', 'Animal Type', 'Flag results', 'Model_type', 'Flag', 'Predictions']]

# Count the occurrences of each unique value in 'Flag results'
flag_results_counts = autoTS_results_df['Flag results'].value_counts()

# Display the counts
print(flag_results_counts)


Flag results
AutoTS    190
Name: count, dtype: int64


In [180]:

# Assuming your CSV file is in the current directory or you provide the full path
excel_file = 'AUTOTS_evaluation_results_ensemble_all.xlsx'
#Read the CSV file into a DataFrame
results_autots_ensemble_df = pd.read_excel(excel_file)

#for the autoTS-with esemble we keep all the results, the ones that failed and nto as we are not going to try more models
autoTS_esemble_results_df = results_autots_ensemble_df[['Country', 'Animal Type', 'Flag results', 'Model_type', 'Flag', 'Predictions']]

# Count the occurrences of each unique value in 'Flag results'
flag_results_counts = autoTS_esemble_results_df['Flag results'].value_counts()

# Display the counts
print(flag_results_counts)


Flag results
AutoTS-ensemble    279
Unknown Model        6
Name: count, dtype: int64


### Merge all result dataframes

In [181]:
# Concatenate the DataFrames
results_df= pd.concat([autoTS_esemble_results_df, autoTS_results_df, arima_results_df], ignore_index=True)
# Specify the path where you want to save the Excel file
excel_file = 'final_results_df_concatenated.xlsx'

# Save the DataFrame to Excel
results_df.to_excel(excel_file, index=False)
results_df

,Country,Animal Type,Flag results,Model_type,Flag,Predictions
0,Qatar,Sheep,AutoTS-ensemble,NaN,AUTOTS-InsideCINotMet,LASTVALUE
1,Ireland,Goats,AutoTS-ensemble,AutoTS BestN,AUTOTS-InsideCINotMet,LASTVALUE
2,China,Cattle,AutoTS-ensemble,AutoTS BestN,AUTOTS-InsideCINotMet,LASTVALUE
3,New Caledonia,Sheep,AutoTS-ensemble,AutoTS BestN,AUTOTS-InsideCINotMet,LASTVALUE
4,Czechia,Goats,AutoTS-ensemble,NaN,AUTOTS-InsideCINotMet,LASTVALUE
...,...,...,...,...,...,...
1163,Yugoslav SFR,Buffalo,NoTestData,NaN,NoTestData,LASTVALUE
1164,Yugoslav SFR,Cattle,NoTestData,NaN,NoTestData,LASTVALUE
1165,Yugoslav SFR,Horses,NoTestData,NaN,NoTestData,LASTVALUE
1166,Yugoslav SFR,Sheep,NoTestData,NaN,NoTestData,LASTVALUE


#### Count the porportion of models  where we couldn't get a "model"prediction

In [182]:
# Count the occurrences of each unique value in 'Predictions'
predictions_counts = results_df['Predictions'].value_counts()

# Calculate the percentage of each unique value
percentage_counts = (predictions_counts / len(results_df)) * 100

# Display the percentage counts
print(percentage_counts)

Predictions
MODEL        72.174658
LASTVALUE    27.825342
Name: count, dtype: float64


In [183]:
# Count the occurrences of each unique value in 'Predictions'
predictions_counts = results_df['Flag results'].value_counts()

# Calculate the percentage of each unique value
percentage_counts = (predictions_counts / len(results_df)) * 100

# Display the percentage counts
print(percentage_counts)

Flag results
AutoARIMA          51.284247
AutoTS-ensemble    23.886986
AutoTS             16.267123
NoTestData          4.109589
No Data             3.852740
Unknown Model       0.513699
constant            0.085616
Name: count, dtype: float64


In [184]:
# Count the occurrences of each unique value in 'Predictions'
predictions_counts = results_df['Flag'].value_counts()

# Calculate the percentage of each unique value
percentage_counts = (predictions_counts / len(results_df)) * 100

# Display the percentage counts
print(percentage_counts)

Flag
AutoARIMA                51.284247
AUTOTS-InsideCINotMet    15.410959
AutoTS-ARIMA              8.390411
AutoTS-ETS                7.363014
AutoTS-BestN              4.452055
NoTestData                4.109589
AutoTS-nan                4.023973
No Data                   3.852740
Unknown Model-nan         0.513699
AutoTS-FBProphet          0.513699
constant                  0.085616
Name: count, dtype: float64


### Perform forecasting  code

In [185]:

# Split into two DataFrames based on 'Predictions'
unique_combinations_modelling = results_df[results_df['Predictions'] == 'MODEL'][['Country', 'Animal Type']].drop_duplicates()
unique_combinations_last_value = results_df[results_df['Predictions'] == 'LASTVALUE'][['Country', 'Animal Type']].drop_duplicates()

In [186]:
# Find duplicate rows based on 'Country' and 'Animal Type'

results_df

,Country,Animal Type,Flag results,Model_type,Flag,Predictions
0,Qatar,Sheep,AutoTS-ensemble,NaN,AUTOTS-InsideCINotMet,LASTVALUE
1,Ireland,Goats,AutoTS-ensemble,AutoTS BestN,AUTOTS-InsideCINotMet,LASTVALUE
2,China,Cattle,AutoTS-ensemble,AutoTS BestN,AUTOTS-InsideCINotMet,LASTVALUE
3,New Caledonia,Sheep,AutoTS-ensemble,AutoTS BestN,AUTOTS-InsideCINotMet,LASTVALUE
4,Czechia,Goats,AutoTS-ensemble,NaN,AUTOTS-InsideCINotMet,LASTVALUE
...,...,...,...,...,...,...
1163,Yugoslav SFR,Buffalo,NoTestData,NaN,NoTestData,LASTVALUE
1164,Yugoslav SFR,Cattle,NoTestData,NaN,NoTestData,LASTVALUE
1165,Yugoslav SFR,Horses,NoTestData,NaN,NoTestData,LASTVALUE
1166,Yugoslav SFR,Sheep,NoTestData,NaN,NoTestData,LASTVALUE


In [136]:
# Define the number of combinations to select
num_combinations =1
unique_combinations = results_df[['Country', 'Animal Type']].drop_duplicates()

# Randomly select a subset of unique country-animal combinations
random_combinations = unique_combinations.sample(num_combinations, replace=False, random_state=123)
evaluation_results = {}

# Initialize a dictionary to store error information
error_log = {}

# Initialize an empty DataFrame to store forecasted results
#forecast_df = pd.DataFrame(columns=['Country', 'Animal Type', 'Year', 'Forecasted Value', 'Lower CI', 'Upper CI', 'Model Type', 'Prediction', 'Flag'])
forecast_data = []

# Iterate over unique_combinations
with PdfPages('forecasted_values.pdf') as pdf:
    for index, row in unique_combinations.iterrows():
        title_color = 'red'
        country = row['Country']
        animal_type = row['Animal Type']
        
        # Filter the DataFrame for the specific country and animal type combination
        filtered_df = data_df[(data_df['Area'] == country) & (data_df['Item'] == animal_type)]
        results_filtered_df = results_df[(results_df['Country'] == country) & (results_df['Animal Type'] == animal_type)]
        forecast_years = range(START_FORECASTING_YEAR, START_FORECASTING_YEAR + FORECASTING_LENGTH)

        
        try:     
            if results_filtered_df['Predictions'].item() == 'MODEL':
                 
                # Evaluate models based on model_type
                if 'AutoTS' in results_filtered_df['Model_type'].item():
                    print('AutoTS')
                    
                    if 'AutoTS-ensemble' in results_filtered_df['Flag results'].item():
                        model_type, selected_model, constant_prediction, prediction, conf_int  = forecast_auto_ts(data_df, country, animal_type, ensemble='simple')
                        predictions = prediction['Value']
                        model_type += '-ensemble'  # Append '-ensemble' to model_type
                    else:
                        model_name = results_filtered_df['Model_type'].item().split(' ')[1]
                        model_type, selected_model, constant_prediction, prediction, conf_int  = forecast_auto_ts(data_df, country, animal_type, model_list=[model_name], ensemble=None)
                        predictions = prediction['Value']
                elif 'AutoARIMA' in results_filtered_df['Flag results'].item():
                    print('AutoARIMA')
                    model_type, selected_model, constant_prediction, predictions, conf_int = forecast_arima(data_df, country, animal_type)
                    
                else:
                    raise ValueError("No valid model type found.")
                    
            elif results_filtered_df['Predictions'].item() == 'LASTVALUE':
                last_value = filtered_df.loc[filtered_df['Year'].idxmax(), 'Value']  # Get the last value available
                predictions = [last_value] * FORECASTING_LENGTH
                conf_int = None
                model_type = results_filtered_df['Flag'].item()
                selected_model = results_filtered_df['Flag results'].item()
                constant_prediction = True

            # Prepare data for forecasted values from 2022 to 2034
           
            forecast_values = predictions
            if conf_int is not None:
                lower_ci = conf_int[:, 0]
                upper_ci = conf_int[:, 1]
            else:
                lower_ci = [None] * FORECASTING_LENGTH  # Set None for lower CI
                upper_ci = [None] * FORECASTING_LENGTH  # Set None for upper CI

            # Append forecasted values to forecast_data
            for year, forecast_val, lc, uc in zip(forecast_years,forecast_values, lower_ci, upper_ci):
                forecast_data.append({
                    'Country': country,
                    'Animal Type': animal_type,
                    'Year': year,
                    'Forecasted Value': forecast_val,
                    'Lower CI': lc,
                    'Upper CI': uc,
                    'Model Type': selected_model,
                    'Prediction': results_filtered_df['Predictions'].item(),
                    'Flag': model_type,
                })
        
            # Plot actual vs. predicted values (example plot)
            plt.figure(figsize=(8, 4))
            plt.plot(filtered_df['Year'], filtered_df['Value'], label='Actual Data')

            if predictions is not None:
                plt.plot(forecast_years, forecast_values, label='Forecasted Data')
                # Plot confidence intervals ensuring first that they are not None
                if conf_int is not None and not constant_prediction:
                    title_color = 'green'
                    plt.fill_between(forecast_years, conf_int[:, 0], conf_int[:, 1], color='gray', alpha=0.2, label='Confidence Interval')

            plt.xlabel('Year')
            plt.ylabel('Population')
            title_text_obj = plt.title(f'{selected_model} prediction for {country} - {animal_type}')
            title_text_obj.set_color(title_color)
            plt.legend()
            plt.grid(True)

            # Save the current plot to the PDF
            pdf.savefig()
            plt.close()
                
        except Exception as e:
            # Log the error and continue to the next iteration
            error_log[(country, animal_type)] = str(e)
             # Plot actual vs. predicted values
            plt.figure(figsize=(8, 4))
            plt.plot(filtered_df['Year'], filtered_df['Value'], label='Actual Data')
            plt.xlabel('Year')
            plt.ylabel('Population')
            title_text_obj = plt.title(f'{selected_model} prediction for {country} - {animal_type}')
            title_text_obj.set_color(title_color)
            plt.legend()
            plt.grid(True)

            # Save the current plot to the PDF
            pdf.savefig()
            plt.close()
            continue
# Convert forecast_data list to DataFrame
forecast_df = pd.DataFrame(forecast_data)
# After the loop, you can inspect the error_log if needed
print("\nErrors encountered:")
for key, error_message in error_log.items():
    print(f"For {key}: {error_message}")



AutoTS
Using 3 cpus for n_jobs.

Errors encountered:


<b> Create predictions dataframe </b>

In [98]:
#create predictions dataframe
# Convert forecast_data list to DataFrame
forecast_df = pd.DataFrame(forecast_data)

# Specify the file path where you want to save the Excel file
file_path = 'forecast_data.xlsx'

# Save the DataFrame to Excel
forecast_df.to_excel(file_path, index=False)
forecast_df

,Country,Animal Type,Year,Forecasted Value,Lower CI,Upper CI,Model Type,Prediction,Flag
0,Qatar,Sheep,2024,813941.0,NaN,NaN,AutoTS-ensemble,LASTVALUE,AUTOTS-InsideCINotMet
1,Qatar,Sheep,2025,813941.0,NaN,NaN,AutoTS-ensemble,LASTVALUE,AUTOTS-InsideCINotMet
2,Qatar,Sheep,2026,813941.0,NaN,NaN,AutoTS-ensemble,LASTVALUE,AUTOTS-InsideCINotMet
3,Qatar,Sheep,2027,813941.0,NaN,NaN,AutoTS-ensemble,LASTVALUE,AUTOTS-InsideCINotMet
4,Qatar,Sheep,2028,813941.0,NaN,NaN,AutoTS-ensemble,LASTVALUE,AUTOTS-InsideCINotMet
...,...,...,...,...,...,...,...,...,...
9155,Yugoslav SFR,Swine / pigs,2029,7378429.0,NaN,NaN,NoTestData,LASTVALUE,NoTestData
9156,Yugoslav SFR,Swine / pigs,2030,7378429.0,NaN,NaN,NoTestData,LASTVALUE,NoTestData
9157,Yugoslav SFR,Swine / pigs,2031,7378429.0,NaN,NaN,NoTestData,LASTVALUE,NoTestData
9158,Yugoslav SFR,Swine / pigs,2032,7378429.0,NaN,NaN,NoTestData,LASTVALUE,NoTestData
